# Đồ án cuối kỳ môn: Lập trình cho khoa học dữ liệu

## Danh sách thành viên

| MSSV  | Họ tên  | Tài khoản github  |
|:------|:--------|:------------------|
1712770| Trương Thị Lệ Thanh  |thanhtruong1712|
1712771|  Bùi Thái Tấn Thành  |thanhtan0501|

## Import các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

## Thu thập dữ liệu

Bộ dữ liệu cung cấp thông tin về tổng số ca mắc, tử vong và hồi phục do tác động của dịch COVID-19 từ các quốc gia trên thế giới
- Nguồn dữ liệu: https://www.kaggle.com/imdevskp/corona-virus-report

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [2]:
df = pd.read_csv("Data/covid_19_clean_complete.csv")
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


### Số dòng và số cột của dữ liệu

Số dòng và số cột của dữ liệu sẽ được tính và lưu vào hai biến `num_rows` và `num_cols`.

In [3]:
num_rows, num_cols = df.shape
print("Dữ liệu có {} dòng và {} cột".format(num_rows, num_cols))

Dữ liệu có 49068 dòng và 10 cột


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

### Dữ liệu có các dòng bị lặp không?

### Mỗi cột có ý nghĩa gì?

### Mỗi cột hiện đang có kiểu dữ liệu gì?

### Với mỗi cột có kiểu dữ liệu dạng số (numerical), các giá trị được phân bố như thế nào?

### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào? 

## Đưa ra câu hỏi cần trả lời

### 1. Top 10 quốc gia có số ca nhiễm cao nhất thế giới ? 

Trả lời được câu hỏi này, ta sẽ biết được tình hình kiểm soát dịch của các quốc gia trên thế giới như thế nào.

### 2. Mỗi tháng, số ca tử vong của quốc gia nào cao nhất?

Trả lời được câu hỏi này, ta sẽ biết được khả năng phòng chống dịch của một quốc gia có tốt hay không , các biện pháp hỗ trợ người dân khi bị mắc COVID-19 có đầy đủ hay không?